In [1]:
import numpy as np, torch, os, glob, nibabel as nib, multiprocessing, pandas as pd
import torch.backends.cudnn as cudnn, torchio as tio, random
import segmentation_models_pytorch as smp
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import albumentations as A
from sklearn.metrics import *
from utils.model_res import generate_model
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
from scipy import ndimage
from tqdm import tqdm   

num_workers = multiprocessing.cpu_count()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True
random.seed(1234)
torch.manual_seed(1234)

In [2]:
# load csv label
if True:
    csv_path = './20211104_label_1-350.csv'
    table =  pd.read_csv(csv_path)
    table_3t = table[table['1/0: 3T/1.5T MRI']==1.0]
    table_3t_test = table_3t[table_3t['Valid data']=='V']
    table_3t_test = np.array(table_3t_test[table_3t_test['排除']=='Test data'])
    nii_3t_test = sorted([i for i in os.listdir(os.path.join('./dataset/S2_data/','test'))])

In [3]:
# Subject Function Building
def tio_process(nii_3t_, table_3t_, basepath_='./dataset/S2_data/train/'):
    subjects_ = []
    for  (nii_path, nii_table) in zip(nii_3t_ , table_3t_):
        if (params['S2_type']=='ap') and (nii_table[3]=='A' or nii_table[3]=='P'):
            subject = tio.Subject(
                dwi = tio.ScalarImage(os.path.join(basepath_, nii_path)), 
                ap = nii_table[3], 
                score=[])
            subjects_.append(subject)
        elif (params['S2_type']=='nl'):
            subject = tio.Subject(
                dwi = tio.ScalarImage(os.path.join(basepath_, nii_path)), 
                nl  = nii_table[4], 
                score=[])
            subjects_.append(subject)
    return subjects_


In [ ]:
def S1_evaluate(model, valid_loader):
    predict_array = []
    model.eval()
    stream = tqdm(valid_loader)
    with torch.no_grad():
        for i, (images, target) in enumerate(stream, start=1):
            images = images.to('cpu')
            target = target.to('cpu')
            output =  model(images).squeeze(1)
            predict_array.append(output.detach().numpy()>0.3)
    return 0

def S2_evaluate():

    return 0

S1_weight = './checkpoint/2021.11.03.t1 - 2DRes18Unet/best-2DRes18Unet - lr_0.003 - DCEL.pt'
S2_weight_ap = './checkpoint/2021.11.11.t6 - 3DResNet18 - ap/best - ap - 3dresnet18 - lr_0.001 - CE.pt'
S2_weight_nl = './checkpoint/2021.11.10.t4 - 3DResNet18 - nl/best - nl - 3dresnet18 - lr_0.003 - CE.pt'

S1_checkpoint = torch.load(S1_weight, map_location=torch.device('cpu'))
S1_model = smp.Unet(encoder_name='resnet18', encoder_weights=None, in_channels=1, classes=1)
S1_model.load_state_dict(S1_checkpoint['model_state_dict'])
S1_model.to('cpu')
for idx, i in enumerate(['ap', 'nl']):
    params = {"S1_type": None, "S2_type": i}
    test_subjects = tio_process(nii_3t_test, table_3t_test, basepath_ = './dataset/S2_data/test/')
    S1_evaluate()
    S2_evaluate()

